In [1]:
import sys
import pandas as pd
import os
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
sys.path.append("../../../data_pipeline")
sys.path.append("../..")
sys.path.append("..")

from utils import train_test_split, create_metadata_df
from dataset_xy import CARLADatasetXY
from dataset_xy_opt import CARLADatasetXYOpt
from data_preprocessing import preprocessing

from baseline_v1 import Baseline_V1
from baseline_v2 import Baseline_V2

from model_trainer import ModelTrainer

/Users/julianvonklitzing/miniforge3/envs/carla/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Create Dataset and DataLoader

In [2]:
path_data = os.path.join("..", "..", "..", "data", "data")
# path_data = os.path.join("..", "..", "..", "data", "data_prep")

config_xy = {"used_inputs": ["rgb", "measurements"], 
        "used_measurements": ["speed", "steer", "throttle", "brake", "command"],
        "y": ["brake", "steer", "throttle"],
        "seq_len": 1
        }

In [3]:
batch_size = 64

# Create df_meta 
df_meta_data = create_metadata_df(path_data, config_xy["used_inputs"])
df_meta_data_train, df_meta_data_test = train_test_split(df_meta_data, towns={"train": ["Town04"], "test": ["Town05"]}) # "Town06"
# Make test set the 15% of the size of the train set
df_meta_data_test = df_meta_data_test.sample(n=int(df_meta_data_train.shape[0] * 0.15), random_state=3)

# df_meta_data_train = df_meta_data # test whole set for nan values
# df_meta_data_test = df_meta_data.sample(n=int(df_meta_data_train.shape[0] * 0.15), random_state=3)
# df_meta_data_train = df_meta_data_train.head(5 * batch_size)
# df_meta_data_test = df_meta_data_test.sample(n=int(df_meta_data_train.shape[0] * 0.15), random_state=3)

# Create Dataset & DataLoader
dataset_train = CARLADatasetXY(root_dir=path_data, df_meta_data=df_meta_data_train, config=config_xy)
dataset_test = CARLADatasetXY(root_dir=path_data, df_meta_data=df_meta_data_test, config=config_xy)
# dataset_train = CARLADatasetXYOpt(df_meta_data_train)
# dataset_test = CARLADatasetXYOpt(df_meta_data_test)

dataloader_train = DataLoader(dataset_train, batch_size=batch_size, num_workers=0, shuffle=True)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, num_workers=0, shuffle=False)

In [ ]:
len(dataloader_train)

952

In [ ]:
len(dataloader_test)

143

# Create ModelTrainer & run it

In [ ]:
model=Baseline_V1()

/Users/julianvonklitzing/miniforge3/envs/carla/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/julianvonklitzing/miniforge3/envs/carla/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# see fix for multiple inputs: https://github.com/sksq96/pytorch-summary/issues/90
# summary(model, input_size=[(3, 160, 960), (7,), (1,)], batch_size=32)

In [ ]:
model_trainer = ModelTrainer(
    model=model,
    optimizer=optim.Adam(model.parameters(), lr=0.0001),
    loss_fn=nn.L1Loss(),
    n_epochs=10,
    dataloader_train=dataloader_train,
    dataloader_test=dataloader_test,
    preprocessing=preprocessing,
    upload_tensorboard=False
    )

Model will be trained on: mps


In [ ]:
model_trainer.run()

Epoch 1



/Users/julianvonklitzing/miniforge3/envs/carla/lib/python3.8/site-packages/torch/autograd/__init__.py:197: UserWarning: The operator 'aten::sgn.out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch [1/10], Step [0/952], Loss: 0.5308
Epoch [1/10], Step [200/952], Loss: 0.1513
Epoch [1/10], Step [400/952], Loss: 0.1559
Epoch [1/10], Step [600/952], Loss: 0.1291
Epoch [1/10], Step [800/952], Loss: 0.1480

train-loss: 0.1469,
Validation Loss: 0.1980, 

Epoch 2

Epoch [2/10], Step [0/952], Loss: 0.1277
Epoch [2/10], Step [200/952], Loss: 0.1472
Epoch [2/10], Step [400/952], Loss: 0.1476


In [ ]:
# model_trainer.df_performance_stats

In [ ]:
"JLKEJDF".lower()

'jlkejdf'

In [ ]:
f"dfklg{46}".lower()

'dfklg46'